In [1]:
spark

In [65]:
jsonRDD1 = spark.read.format("json").load("c:/data/all-world-cup-players.json")

DataFrame[_corrupt_record: string]

In [17]:
jsonRDD = sc.wholeTextFiles("file///c:/data/all-world-cup-players.json",4).map(lambda x: x[1])

In [26]:
sc.wholeTextFiles("file///c:/data/all-world-cup-players.json").take(1)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [18]:
import re
js = jsonRDD.map(lambda x: re.sub(r"\s+", "", x, flags=re.UNICODE))

In [22]:
wc_players = sqlContext.read.json(js)

In [11]:
wc_players.printSchema()

root
 |-- Club: string (nullable = true)
 |-- ClubCountry: string (nullable = true)
 |-- Competition: string (nullable = true)
 |-- DateOfBirth: string (nullable = true)
 |-- FullName: string (nullable = true)
 |-- IsCaptain: boolean (nullable = true)
 |-- Number: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- Year: long (nullable = true)



### https://stackoverflow.com/questions/43551670/pyspark-json-object-or-file-to-rdd

In [27]:
askRedditDF = spark.read.json("c:/data/AskReddit.json")

In [28]:
askRedditDF.printSchema()

root
 |-- _corrupt_record: string (nullable = true)



In [29]:
redditRDD = sc.wholeTextFiles("file///c:/data/AskReddit.json",4).map(lambda x: x[1])

In [30]:
import re
redditjs = redditRDD.map(lambda x: re.sub(r"\s+", "", x, flags=re.UNICODE))

In [31]:
askRedditDF = sqlContext.read.json(redditjs)

In [32]:
askRedditDF.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- after: string (nullable = true)
 |    |-- before: string (nullable = true)
 |    |-- children: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- data: struct (nullable = true)
 |    |    |    |    |-- approved_at_utc: string (nullable = true)
 |    |    |    |    |-- approved_by: string (nullable = true)
 |    |    |    |    |-- archived: boolean (nullable = true)
 |    |    |    |    |-- author: string (nullable = true)
 |    |    |    |    |-- author_flair_css_class: string (nullable = true)
 |    |    |    |    |-- author_flair_text: string (nullable = true)
 |    |    |    |    |-- banned_at_utc: string (nullable = true)
 |    |    |    |    |-- banned_by: string (nullable = true)
 |    |    |    |    |-- brand_safe: boolean (nullable = true)
 |    |    |    |    |-- can_gild: boolean (nullable = true)
 |    |    |    |    |-- can_mod_post: boolean (nullable = true)
 |    |    |    |  

In [34]:
askRedditDF
.withColumn('after',askRedditDF.data.after)\
.withColumn('before',askRedditDF.data.before)\
.withColumn('modhash',askRedditDF.data.modhash)\
.withColumn('NestedKind',explode(askRedditDF.data.children.kind))\
.withColumn('subreddit',explode(askRedditDF.data.children.data.subreddit))\
.withColumn('clicked',explode(askRedditDF.data.children.data.clicked))\
.show()

SyntaxError: invalid syntax (<ipython-input-34-f6d04d4591b7>, line 2)

In [40]:
from pyspark.sql.functions import explode

askRedditDF.withColumn('after',askRedditDF.data.after).withColumn('before',askRedditDF.data.before).withColumn('modhash',askRedditDF.data.modhash).withColumn('NestedKind',explode(askRedditDF.data.children.kind)).withColumn('clicked',explode(askRedditDF.data.children.data.clicked)).show()

+--------------------+-------+---------+------+-------+----------+-------+
|                data|   kind|    after|before|modhash|NestedKind|clicked|
+--------------------+-------+---------+------+-------+----------+-------+
|[t3_7m1hy4,null,W...|Listing|t3_7m1hy4|  null|       |        t3|  false|
|[t3_7m1hy4,null,W...|Listing|t3_7m1hy4|  null|       |        t3|  false|
|[t3_7m1hy4,null,W...|Listing|t3_7m1hy4|  null|       |        t3|  false|
|[t3_7m1hy4,null,W...|Listing|t3_7m1hy4|  null|       |        t3|  false|
|[t3_7m1hy4,null,W...|Listing|t3_7m1hy4|  null|       |        t3|  false|
|[t3_7m1hy4,null,W...|Listing|t3_7m1hy4|  null|       |        t3|  false|
|[t3_7m1hy4,null,W...|Listing|t3_7m1hy4|  null|       |        t3|  false|
|[t3_7m1hy4,null,W...|Listing|t3_7m1hy4|  null|       |        t3|  false|
|[t3_7m1hy4,null,W...|Listing|t3_7m1hy4|  null|       |        t3|  false|
|[t3_7m1hy4,null,W...|Listing|t3_7m1hy4|  null|       |        t3|  false|
|[t3_7m1hy4,null,W...|Lis

### https://docs.databricks.com/spark/latest/spark-sql/complex-types.html

In [41]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Convenience function for turning JSON strings into DataFrames.
def jsonToDataFrame(json, schema=None):
  # SparkSessions are available with Spark 2.0+
  reader = spark.read
  if schema:
    reader.schema(schema)
  return reader.json(sc.parallelize([json]))

In [45]:
# Using a struct
schema = StructType().add("a", StructType().add("b", IntegerType()))
                          
events = jsonToDataFrame("""
{
  "a": {
     "b": 1,
     "c": 1
  }
}
""", schema)


In [46]:
events.show()

+---+
|  a|
+---+
|[1]|
+---+



In [47]:
# Using a map
schema = StructType().add("a", MapType(StringType(), IntegerType()))
                          
events = jsonToDataFrame("""
{
  "a": {
     "b": 1
  }
}
""", schema)

In [48]:
events.show()

+-----------+
|          a|
+-----------+
|Map(b -> 1)|
+-----------+



In [49]:
events = jsonToDataFrame("""
{
  "a": 1,
  "b": 2,
  "c": 3
}
""")

In [50]:
events.show()

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  2|  3|
+---+---+---+



In [51]:
events = jsonToDataFrame("""
{
  "a": 1,
  "b": 2
}
""")

In [54]:
events.select(struct("*").alias("x")).show()

+-----+
|    x|
+-----+
|[1,2]|
+-----+



In [55]:
# Using a map
schema = StructType().add("a", MapType(StringType(), IntegerType()))

events = jsonToDataFrame("""
{
  "a": {
    "b": 1
  }
}
""", schema)

In [56]:
events.select(col("a").getItem("b").alias("x")).show()

+---+
|  x|
+---+
|  1|
+---+



In [57]:
events = jsonToDataFrame("""
{
  "a": [1, 2]
}
""")

events.select(explode("a").alias("x")).show()

+---+
|  x|
+---+
|  1|
|  2|
+---+



In [58]:
# Using a map
schema = StructType().add("a", MapType(StringType(), IntegerType()))

events = jsonToDataFrame("""
{
  "a": {
    "b": 1,
    "c": 2
  }
}
""", schema)

events.select(explode("a").alias("x", "y")).show()

+---+---+
|  x|  y|
+---+---+
|  b|  1|
|  c|  2|
+---+---+

